# Data reducing and preproccessing
(AMEX example)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
AMEX_TRAIN_DATA_PATH = 'train_data.csv.zip'
AMEX_TRAIN_LABELS_PATH = 'train_labels.csv.zip'

HM_TRAIN_DATA_PATH = 'train_data.csv.zip'

## Calculate data size

In [ ]:
train = pd.read_csv(AMEX_TRAIN_DATA_PATH, usecols=['customer_ID'])
train_len = len(train)

print(f'There are {train_len} rows in train.')
del train

## Spot room for improvment

In [ ]:
train =  pd.read_csv(AMEX_TRAIN_DATA_PATH, nrows=100000)
train[:10]

In [ ]:
train.info(verbose=True, show_counts=True)

In [ ]:
null_cols = train.loc[:, train.isnull().mean() > .90].columns.to_list()
null_cols

In [ ]:
train[['customer_ID','S_2','D_63', 'D_64']][:10]

In [ ]:
train.customer_ID.iloc[0]

In [ ]:
train.nunique()[105:120]

In [ ]:
train[['B_30', 'D_87', 'B_31']][:10]

## Drop null columns 

In [ ]:
train = train.drop(columns=null_cols)

## Convert dates

In [ ]:
train.S_2 = pd.to_datetime(train.S_2)

train['year'] = (train.S_2.dt.year - 2000).astype('int8')
train['month'] = (train.S_2.dt.month).astype('int8')
train['day'] = (train.S_2.dt.day).astype('int8')

del train['S_2']

In [ ]:
train[['year', 'month', 'day']][:5]

## Convert categories to int

**!! Remember that we have been looking at only 100000 rows !!**

In [ ]:
train_b_30 = pd.read_csv(AMEX_TRAIN_DATA_PATH, usecols=['B_30'])

train_b_30.describe().astype('category')

In [ ]:
del train_b_30

In [ ]:
categories = ['B_30', 'B_31']

for c in categories:
    train[c] = train[c].fillna(-2).astype('int8') # int cannot have na!

## Convert string categories

In [ ]:
train.D_63.value_counts()

In [ ]:
d_63_map = {'CL':2, 'CO':3, 'CR':4, 'XL':5, 'XM':6, 'XZ':7}

train.D_63 = train.D_63.map(d_63_map).fillna(-2).astype('int8')

In [ ]:
train.D_63 = train.D_63.astype('category')

**!! During training, ensure that algorithm does not treat them as ordered !!**

## Reduce float size

**Float Downcasting Value Range:**
- float16 - Half precision float : sign bit, 5 bits exponent, 10 bits mantissa
- float32 - Single precision float: sign bit, 8 bits exponent, 23 bits mantissa
- float64 - Double precision float: sign bit, 11 bits exponent, 52 bits mantissa

In [ ]:
for col in train.columns:
    if str(train[col].dtype) =='float64':
        train[col] = train[col].astype('float32')

**Int Downcasting Value Range**:
- int 8 can store ints from -128 to 128
- int 16 can store ints from -32768 to 32768
- int 32 can store ints from -2147483648 to 2147483648
- int 64 can store ints from -9223372036854775808 to 9223372036854775808

## Hex to int

In [ ]:
targets = pd.read_csv(AMEX_TRAIN_LABELS_PATH)
print(f'There are {len(targets)} train targets')

In [ ]:
targets.info()

In [ ]:
print(f'There are {len(targets.customer_ID.unique())} unique train targets')

In [ ]:
targets[:5]

In [ ]:
targets.customer_ID = targets.customer_ID.apply(lambda x: int(x[-16:],16) ).astype('int64')
targets.target = targets.target.astype('int8')

print(f'There are {len(targets.customer_ID.unique())} unique short train targets')

In [ ]:
targets.info()

In [ ]:
targets.to_csv('train_labels_short.csv.zip', index=False, compression={'method': 'zip', 'archive_name': 'train_labels_short.csv'})
del targets

'train_labels.csv.zip' size: 17 MB <br>
'train_labels_short.csv.zip' size: 5.9 MB

In [ ]:
train['customer_ID'] = train['customer_ID'].apply(lambda x: int(x[-16:], 16) ).astype('int64')

## Save options

In [ ]:
# pip install fastparquet
# pip install pyarrow

In [ ]:
train.to_csv('train_part.csv.zip', index=False) # looses dtypes

train.to_pickle('train_part.pickle') 
train.to_parquet('train_part.parquet')
train.to_feather('train_part.feather')

np.save('train_part', train)

In [ ]:
train_part.csv.zip 126.5 MB
train_part.feather 127.7 MB
train_part.pkl     137.7 MB
train_part.parquet 150.7 MB
train_part.npy     154.9 MB

## Load data by chunks

In [ ]:
# You can also make sure that you don't split customers

chunksize = int(train_len / 5)

for index, chunk in enumerate(pd.read_csv(AMEX_TRAIN_DATA_PATH, chunksize=chunksize), start=1):
    
    # processing is applied here 
    
    np.save(f'train_{index}', chunk)
    

In [ ]:
train.info()

Materials used:
https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054
